In [2]:
from tkinter import *
import miller_rabin as mlr_rab
  
def clicked():
    num = int(txt.get())
    res = str(mlr_rab.miller_rabin(num))
    lbl_res.configure(text=res)

window = Tk()  
window.title("Prime number checker")  
window.geometry('400x250')  
lbl = Label(window, text="Enter the num:")  
lbl.grid(column=0, row=0)  
txt = Entry(window,width=10)
txt.grid(column=1, row=0)
txt.focus()
btn = Button(window, text="Enter!", command=clicked)  
btn.grid(column=2, row=0) 
lbl_res = Label(window, text="")
lbl_res.grid(column=0, row=1)  
window.mainloop()